# TCP拥塞控制
## 慢开始
慢开始当中的“慢”并不是指cwnd的增长速率慢，而是在TCP开始发送报文段时先设置cwnd = 1,使得发送方在开始时只发送一一个报文段
- 当cwnd < ssthresh 时使用慢启动
- 当cwnd > ssthresh 时，停止使用慢启动算法，改用拥塞避免算法
## 拥塞避免
让拥塞窗口缓慢的增大，即每经过一个往返时间RTT就把发送方的拥塞窗口cwnd+1

1. TCP连接初始化，将拥塞窗口cwnd设置为1个报文段，即cwnd = 1;
2. 执行慢开始算法，cwnd按指数规律增长，知道cwnd == ssthresh时，开始执行拥塞避免算法，cwnd开始按照线性规律增长；
3. 当网络发生拥塞，把ssthresh值更新为拥塞前ssthresh值的一半，cwnd重新设置为1，再按照2执行；
## 快重传
如果接收方收到三个重复的ACK，那么发送方不必等待重传计时器到期，发送方尽早重传未被确认的报文段
## 快恢复
由于发送方现在认为网络很可能没有发生阻塞，因此现在不执行慢启动算法，而是把cwnd值设置为慢启动门限减半后的值，然后开始执行拥塞避免算法，拥塞窗口cwnd值线性增大。

In [189]:
%matplotlib qt5

from enum import Enum
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm



In [190]:
# 事件
class events(Enum):
    # 丢包，三次重复ACK
    LOSS_PACK = 1
    # 超时
    TIME_OUT = 2
    # 正常
    NORMAL = 3

# 事件处理
class handle_events:
    ssthresh = 0
    cwnd = 0
    cwnd_list=[]
    ssthresh_list =[]

    def __init__(self,ssthresh,cwnd):
        self.ssthresh = ssthresh
        self.cwnd = cwnd

    def handle(self,event:events):
        if event is events.LOSS_PACK: # 发生丢包
            self.ssthresh = self.cwnd/2
            self.cwnd = self.ssthresh+3
            self.print_res('loss')
            return
        elif event is events.NORMAL:
            temp = self.cwnd*2
            if self.cwnd is 0: # 初始为0
                self.cwnd=1
            elif self.cwnd>=self.ssthresh:# 大于阈值，线性增长
                self.cwnd+=1       
            elif temp>=self.ssthresh: # 即将超过阈值时，调整为阈值
                self.cwnd = self.ssthresh
            else: # 翻倍
                self.cwnd*=2
            self.print_res('normal')
            return
        elif event is events.TIME_OUT: # 发生超时事件
            self.ssthresh = self.cwnd/2
            self.cwnd = 1
            self.print_res('timeout')
            return
    def print_res(self,condi):
        self.cwnd_list.append(self.cwnd)
        self.ssthresh_list.append(self.ssthresh)
        print(condi.ljust(10,' ') + f' ssthresh: {self.ssthresh} cwnd: {self.cwnd}')
    
    def plot_res(self):
        plt.plot(range(len(self.cwnd_list)),self.cwnd_list,"b", linewidth=2.0)

In [191]:
my_events=[
events.NORMAL,events.NORMAL,events.NORMAL,events.NORMAL,events.NORMAL,
events.NORMAL,events.NORMAL,events.NORMAL,events.LOSS_PACK,events.NORMAL,
events.NORMAL,events.NORMAL,events.NORMAL,events.NORMAL,events.NORMAL,
events.TIME_OUT,events.NORMAL,events.NORMAL,events.NORMAL,events.NORMAL,
events.NORMAL,events.LOSS_PACK,events.NORMAL,events.NORMAL,events.NORMAL,
events.NORMAL,events.NORMAL,events.NORMAL,]

In [192]:
# 绘制图像
def my_plot(my_handle:handle_events):
    # 生成画布
    plt.figure(figsize=(10, 8), dpi=80)

    # 打开交互模式
    plt.ion()

    myfont = fm.FontProperties(size=14)

    
    # 循环
    for event in my_events:
        my_handle.handle(event)
        
        lenth = len(my_handle.cwnd_list)
        # 清除原有图像
        plt.cla()

        # 设定标题等
        plt.title("Congrestion Control", fontproperties=myfont)
        plt.grid(True)

        x = list(map(lambda x:x+1,range(lenth)))
        y = my_handle.cwnd_list
        y_thresh = my_handle.ssthresh_list

        # 设置X轴
        plt.xlabel("RTT", fontproperties=myfont)
        plt.xlim(0,30)
        plt.xticks(np.linspace(0,30,31))

        # 设置Y轴
        plt.ylabel("cwnd", fontproperties=myfont)
        plt.ylim(0,20)
        plt.yticks(np.linspace(0,20,21))

        # 画两条曲线
        plt.plot(x, y, c='r',ls='-',marker='o',mec='b',mfc='w', label="cwnd")
        plt.plot(x, y_thresh, c='b',ls='--', label="threshold")

        # 设置图例位置,loc可以为[upper, lower, left, right, center]
        plt.legend(loc="upper left", prop=myfont, shadow=True)

        # 暂停
        plt.pause(0.2)

    # 关闭交互模式
    plt.ioff()

    # 图形显示
    plt.show()
    return
my_handle=handle_events(8,0)
my_plot(my_handle)

normal     ssthresh: 8 cwnd: 1
normal     ssthresh: 8 cwnd: 2
normal     ssthresh: 8 cwnd: 4
normal     ssthresh: 8 cwnd: 8
normal     ssthresh: 8 cwnd: 9
normal     ssthresh: 8 cwnd: 10
normal     ssthresh: 8 cwnd: 11
normal     ssthresh: 8 cwnd: 12
loss       ssthresh: 6.0 cwnd: 9.0
normal     ssthresh: 6.0 cwnd: 10.0
normal     ssthresh: 6.0 cwnd: 11.0
normal     ssthresh: 6.0 cwnd: 12.0
normal     ssthresh: 6.0 cwnd: 13.0
normal     ssthresh: 6.0 cwnd: 14.0
normal     ssthresh: 6.0 cwnd: 15.0
timeout    ssthresh: 7.5 cwnd: 1
normal     ssthresh: 7.5 cwnd: 2
normal     ssthresh: 7.5 cwnd: 4
normal     ssthresh: 7.5 cwnd: 7.5
normal     ssthresh: 7.5 cwnd: 8.5
normal     ssthresh: 7.5 cwnd: 9.5
loss       ssthresh: 4.75 cwnd: 7.75
normal     ssthresh: 4.75 cwnd: 8.75
normal     ssthresh: 4.75 cwnd: 9.75
normal     ssthresh: 4.75 cwnd: 10.75
normal     ssthresh: 4.75 cwnd: 11.75
normal     ssthresh: 4.75 cwnd: 12.75
normal     ssthresh: 4.75 cwnd: 13.75
